In [21]:
from dotenv import load_dotenv
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import render_text_description, Tool
from langchain_classic.agents.output_parsers import ReActSingleInputOutputParser
from langchain_core.agents import AgentAction, AgentFinish
from typing import Union
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage
from langchain_classic.agents.format_scratchpad import format_log_to_str

load_dotenv()

True

In [9]:
@tool
def get_text_length(text: str) -> int:
    """returns the length of the text by characters"""
    text = text.strip("'\n").strip('"')

    return len(text)

In [18]:
tools = [get_text_length]

In [26]:
def find_tool_by_name(tool_input: str) -> Tool:
    for t in tools:
        if tool_input == t.name:
            return t
    
    return ValueError(f"Tool with name {tool_input} is not found")

In [ ]:
if __name__ == "__main__":
    print("Hello React Agent langchain")
    

    template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought: {agent_scratchpad}
    """

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, model_kwargs={"stop": ["\nObservation", "Observation"]})

    prompt_template = PromptTemplate.from_template(template=template).partial(
        tools=render_text_description(tools), tool_names = ", ".join(tool.name for tool in tools)
    )

    output_parser = ReActSingleInputOutputParser()

    agent = {"input": lambda x: x["input"], "agent_scratchpad": lambda x: format_log_to_str(x["agent_scratchpad"])} | prompt_template | llm | output_parser

    intermediate_steps = []

    agent_step = None

    while not isinstance(agent_step, AgentFinish):
        agent_step: Union[AgentAction, AgentFinish] = agent.invoke({"input": "what is the length of 'DOG' in characters?", "agent_scratchpad": intermediate_steps})

        if isinstance(agent_step, AgentAction):
            tool_name = agent_step.tool
            tool_to_use = find_tool_by_name(tool_name)
            tool_input = agent_step.tool_input

            observation = tool_to_use.func(str(tool_input))
            print("f{observation}")
            intermediate_steps.append((agent_step, str(observation)))

    if isinstance(agent_step, AgentFinish):
        print(agent_step.return_values)
        


Hello React Agent langchain
f{observation}
{'output': '3'}
